In [72]:
import sys
sys.path.append('/home/potzschf/repos/')
from helperToolz.helpsters import *
from helperToolz.evapo import *
import geopandas as gpd
import zipfile
from collections import defaultdict
workhorse = True

if workhorse:
    origin = 'Aldhani/eoagritwin/'
else:
    origin = ''

In [73]:
# read csv for valid row_cols for samples to draw. They are based on the share of agriculture (HR Landcover maps) within a S3 pixel
thresh_csv = pd.read_csv(f'/data/{origin}et/Auxiliary/landcover/csv/row_cols.csv')

for col in thresh_csv.columns:
    #print(f'finding indices for {col}')
    nested = [entry.split('_') for entry in thresh_csv[col] if type(entry) == str]
    rows, cols = zip(*nested)
    #print(len(rows))
# only use Thresh50, as all valid row_col will be here. For other THresh sets, a subset from this can be taken
# this are the last instances of rows and cols
rows = [int(row) for row in rows]
cols = [int(col) for col in cols]


/tmp/ipykernel_3098642/2538488.py:2: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  thresh_csv = pd.read_csv(f'/data/{origin}et/Auxiliary/landcover/csv/row_cols.csv')


In [123]:
year = 2018
force_path = '/data/Aldhani/eoagritwin/force/output/S3/2018/X0063_Y0044/TSI/'
files = getFilelist(force_path, '.tif')
# delete all other files
[RasterKiller(file) for file in files if str(year) not in file.split('_')[-1]]
files = getFilelist(force_path, '.tif')
files_annual = [file for file in files if str(year) in file]
indices = list(set([file_annual.split('_')[-3] for file_annual in files_annual]))
indices.sort()

In [124]:
# warp em
#for index in indices:
vals = []
index = indices[0]
index_files = [file_annual for file_annual in files_annual if index in file_annual]
#for index_file in index_files:
warpi = warp_to_template(index_files[0], 
        '/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask.tif',
        mask_path='/data/Aldhani/eoagritwin/et/Auxiliary/S3_tif_GER_maks/powermask.tif',
        outPath='/data/Aldhani/eoagritwin/et/Auxiliary/trash/100_nomask.tif',
        outType=gdal.GDT_Int16)
# vals.append(warpi[rows,cols])
# arr = np.stack(vals)
# mask = np.all(arr == -9999, axis=0)
# small_arr = arr[:, ~mask]

# np.save(f'/data/{origin}et/Auxiliary/dumps_for_training_collecting/{year}_{index}_X0063_Y0044_arr10', small_arr)
# nested_masked = [row_col for row_col, valid in zip(nested, ~mask) if valid]
# np.save(f'/data/{origin}et/Auxiliary/dumps_for_training_collecting/{year}_{index}_X0063_Y0044_rowcols10', nested_masked)

In [97]:
rc100 = np.load(f'/data/{origin}et/Auxiliary/dumps_for_training_collecting/{year}_{index}_X0063_Y0044_arr100.npy')
rc10 = np.load(f'/data/{origin}et/Auxiliary/dumps_for_training_collecting/{year}_{index}_X0063_Y0044_arr10.npy')

In [105]:
print(rc100[0:10,5])
print(rc10[0:10,5])

[3602 3602 3602 3602 3602 3602 3602 3602 3602 3602]
[3622 3622 3622 3622 3622 3622 3622 3622 3622 3622]


In [125]:
ds = gdal.Open(index_file,0)
# Get list of all metadata domains
domains = ds.GetMetadataDomainList()

# Collect metadata from all domains
metadata_all = {}

for domain in domains:
    metadata = ds.GetMetadata(domain)
    metadata_all[domain] = metadata
    print(f"--- Metadata for domain: {domain} ---")
    for key, value in metadata.items():
        print(f"{key}: {value}")
    print()

band = ds.GetRasterBand(1)
# Band-specific metadata
print("Band Metadata:", band.GetMetadata())

# Band-specific statistics and other info
print("Min/Max:", band.GetMinimum(), band.GetMaximum())
print("Scale:", band.GetScale())       # for scaled values
print("Offset:", band.GetOffset())
print("Unit Type:", band.GetUnitType())
print("NoData Value:", band.GetNoDataValue())

--- Metadata for domain: IMAGE_STRUCTURE ---
COMPRESSION: LZW
INTERLEAVE: BAND
PREDICTOR: 2

--- Metadata for domain: FORCE ---
FORCE_description: FORCE Time Series Analysis
FORCE_param: DIR_LOWER: /data/Aldhani/dc/deu/ard/, DIR_HIGHER: /data/Aldhani/eoagritwin/force/output/S3/, DIR_PROVENANCE: /data/Aldhani/eoagritwin/force/output/provenance/, DIR_MASK: NULL, BASE_MASK: NULL, FILE_TILE: NULL, X_TILE_RANGE: 63 63, Y_TILE_RANGE: 44 44, RESOLUTION: 10.000000, BLOCK_SIZE: 0.000000, FILE_OUTPUT_OPTIONS: NULL, OUTPUT_FORMAT: 1, OUTPUT_EXPLODE: 1, OUTPUT_SUBFOLDERS: 1, NTHREAD_READ: 12, NTHREAD_WRITE: 18, NTHREAD_COMPUTE: 24, STREAMING: 1, PRETTY_PROGRESS: 1, SENSORS: 5 6 7, PRODUCT_TYPE_MAIN: BOA, PRODUCT_TYPE_QUALITY: QAI, SPECTRAL_ADJUST: 0, SCREEN_QAI: 0 1 2 3 4 5 10 11, DATE_RANGE: 2017-10-01 2019-02-28, DOY_RANGE: 1 365, REDUCE_PSF: 0, USE_L2_IMPROPHE: 0, ABOVE_NOISE: 3.000000, BELOW_NOISE: 1.000000, INDEX: 10 28 11 16 17 18, STANDARDIZE_TSS: 0, OUTPUT_TSS: 0, FILE_ENDMEM: NULL, SMA_SU

In [140]:
arr = ds.GetRasterBand(1).ReadAsArray()
# Replace nodata with NaN
arr = np.where(arr == -9999, np.nan, arr)

# Invert the log10 transform
real_amplitude = 10 ** (arr-4)

/tmp/ipykernel_3098642/3435070080.py:6: RuntimeWarning: overflow encountered in power
  real_amplitude = 10 ** (arr-4)


In [142]:
arr



array([[6304., 6246., 6380., ..., 1997., 2103., 2165.],
       [6456., 6298., 6364., ..., 2318., 2301., 2254.],
       [6551., 6477., 6356., ..., 2285., 2283., 2201.],
       ...,
       [1546., 1692., 1782., ..., 5228., 4605., 4360.],
       [1509., 1806., 1767., ..., 5045., 4502., 4313.],
       [1327., 1623., 1559., ..., 4923., 4528., 4307.]],
      shape=(3000, 3000))

In [ ]:
print(np.unique(warpi))

plt.imshow(warpi[300:335,580:640])
plt.axis('off')  # Hide axis for cleaner image display

plt.show()

In [ ]:
# Build a dict of dicts: row_col → {year: value}
data_dict = {rc: {} for rc in row_col_list}

for year, values in yearly_values.items():
    for rc, val in zip(row_col_list, values):
        data_dict[rc][year] = val

In [ ]:
# Create a small test array (5 rows, 4 columns)
arr = np.array([
    [-9999, -9999, -9999, -9999],  # all -9999 → should be flagged
    [1, 2, 3, 4],                  # valid
    [-9999, -9999, -9999, -9999],  # all -9999 → should be flagged
    [0, -9999, 5, -9999],          # mixed
    [9, 8, 7, 6]                   # valid
])

# Find row indices where all values are -9999
mask = np.all(arr == -9999, axis=1)
bad_row_indices = np.where(mask)[0]
arr[~mask]
print("Rows with all -9999 values:", bad_row_indices)

Rows with all -9999 values: [0 2]


array([[    1,     2,     3,     4],
       [    0, -9999,     5, -9999],
       [    9,     8,     7,     6]])